### Implementando Perceptron de múltipla camada

Para executar o programa abaixo, baixe o caderno e abra no Jupyter Notebook. Para executar uma célula, clique no play que aparece ao lado esquerdo da célula ou pressione shift + enter.

Além disso, é necessário instalar a biblioteca numpy. Para isso, execute o comando abaixo:

```bash
!pip install numpy
```

In [56]:
import numpy as np
train_data = np.array(
    [
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1]])

target_xor = np.array(
    [
        [0],
        [1],
        [1],
        [0]])

target_nand = np.array(
    [
        [1],
        [1],
        [1],
        [0]])

target_or = np.array(
    [
        [0],
        [1],
        [1],
        [1]])

target_and = np.array(
    [
        [0],
        [0],
        [0],
        [1]])

In [2]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [6]:
import matplotlib.pyplot as plt

In [7]:
# implementação do artigo, usada como inspiração 

class MLP:
    """
    Create a multi-layer perceptron.
    train_data: A 4x2 matrix with the input data.
    target: A 4x1 matrix with expected outputs
    lr: the learning rate. Defaults to 0.1
    num_epochs: the number of times the training data goes through the model
        while training
    num_input: the number of nodes in the input layer of the MLP.
        Should be equal to the second dimension of train_data.
    
    num_hidden: the number of nodes in the hidden layer of the MLP.
    num_output: the number of nodes in the output layer of the MLP.
        Should be equal to the second dimension of target.
    """
    def __init__(self, train_data, target, lr=0.1, num_epochs=100, num_input=2, num_hidden=2, num_output=1):
        self.train_data = train_data
        self.target = target
        self.lr = lr
        self.num_epochs = num_epochs

        # initialize both sets of weights and biases randomly
            # - weights_01: weights between input and hidden layer
            # - weights_12: weights between hidden and output layer
        self.weights_01 = np.random.uniform(size=(num_input, num_hidden))
        self.weights_12 = np.random.uniform(size=(num_hidden, num_output))

        # - b01: biases for the  hidden layer
        # - b12: bias for the output layer
        self.b01 = np.random.uniform(size=(1,num_hidden))
        self.b12 = np.random.uniform(size=(1,num_output))

        self.losses = []

    def update_weights(self):
        
        # Calculate the squared error
        loss = 0.5 * (self.target - self.output_final) ** 2
        print(loss)
        self.losses.append(np.sum(loss))

        error_term = (self.target - self.output_final)

        # the gradient for the hidden layer weights
        grad01 = self.train_data.T @ (((error_term * self._delsigmoid(self.output_final)) * self.weights_12.T) * self._delsigmoid(self.hidden_out))
        print("grad01: ", grad01)
        print(grad01.shape)

        # the gradient for the output layer weights
        grad12 = self.hidden_out.T @ (error_term * self._delsigmoid(self.output_final))

        print("grad12: ", grad12)
        print(grad12.shape)

        # updating the weights by the learning rate times their gradient
        self.weights_01 += self.lr * grad01
        self.weights_12 += self.lr * grad12

        # update the biases the same way
        self.b01 += np.sum(self.lr * ((error_term * self._delsigmoid(self.output_final)) * self.weights_12.T) * self._delsigmoid(self.hidden_out), axis=0)
        self.b12 += np.sum(self.lr * error_term * self._delsigmoid(self.output_final), axis=0)

    def _sigmoid(self, x):
        """
        The sigmoid activation function.
        """
        return 1 / (1 + np.exp(-x))

    def _delsigmoid(self, x):
        """
        The first derivative of the sigmoid function wrt x
        """
        return x * (1 - x)

    def forward(self, batch):
        """
        A single forward pass through the network.
        Implementation of wX + b
        """

        self.hidden_ = np.dot(batch, self.weights_01) + self.b01
        self.hidden_out = self._sigmoid(self.hidden_)

        self.output_ = np.dot(self.hidden_out, self.weights_12) + self.b12
        self.output_final = self._sigmoid(self.output_)

        return self.output_final

    def classify(self, datapoint):
        """
        Return the class to which a datapoint belongs based on
        the perceptron's output for that point.
        """
        datapoint = np.transpose(datapoint)
        if self.forward(datapoint) >= 0.5:
            return 1

        return 0

    def plot(self, h=0.01):
        """
        Generate plot of input data and decision boundary.
        """
        # setting plot properties like size, theme and axis limits
        sns.set_style('darkgrid')
        plt.figure(figsize=(20, 20))

        plt.axis('scaled')
        plt.xlim(-0.1, 1.1)
        plt.ylim(-0.1, 1.1)

        colors = {
            0: "ro",
            1: "go"
        }

        # plotting the four datapoints
        for i in range(len(self.train_data)):
            plt.plot([self.train_data[i][0]],
                     [self.train_data[i][1]],
                     colors[self.target[i][0]],
                     markersize=20)

        x_range = np.arange(-0.1, 1.1, h)
        y_range = np.arange(-0.1, 1.1, h)

        # creating a mesh to plot decision boundary
        xx, yy = np.meshgrid(x_range, y_range, indexing='ij')
        Z = np.array([[self.classify([x, y]) for x in x_range] for y in y_range])

        # using the contourf function to create the plot
        plt.contourf(xx, yy, Z, colors=['red', 'green', 'green', 'blue'], alpha=0.4)

    def train(self):
        """
        Train an MLP. Runs through the data num_epochs number of times.
        A forward pass is done first, followed by a backward pass (backpropagation)
        where the networks parameter's are updated.
        """
        for _ in range(self.num_epochs):
            self.forward(self.train_data)
            self.update_weights()

In [8]:
mlp = MLP(train_data, target_xor, 0.2, 5000)
mlp.train()

[[0.31059043]
 [0.0212183 ]
 [0.0142032 ]
 [0.3487421 ]]
grad01:  [[-0.00864203 -0.01634519]
 [-0.00667708 -0.01251167]]
(2, 2)
grad12:  [[-0.12360144]
 [-0.12686825]]
(2, 1)
[[0.3014563 ]
 [0.02372353]
 [0.01615394]
 [0.33941757]]
grad01:  [[-0.00842526 -0.01633762]
 [-0.00640913 -0.01232033]]
(2, 2)
grad12:  [[-0.12448214]
 [-0.12759438]]
(2, 1)
[[0.29207685]
 [0.02648825]
 [0.01834957]
 [0.32969845]]
grad01:  [[-0.00814949 -0.01623509]
 [-0.00609348 -0.01204419]]
(2, 2)
grad12:  [[-0.12475828]
 [-0.12769984]]
(2, 1)
[[0.28251566]
 [0.02951685]
 [0.0208025 ]
 [0.31964551]]
grad01:  [[-0.00781611 -0.01603187]
 [-0.0057332  -0.01168122]]
(2, 2)
grad12:  [[-0.12436609]
 [-0.12712393]]
(2, 1)
[[0.27284644]
 [0.03280671]
 [0.02351944]
 [0.30933466]]
grad01:  [[-0.00742904 -0.01572565]
 [-0.00533352 -0.0112326 ]]
(2, 2)
grad12:  [[-0.12325886]
 [-0.12582391]]
(2, 1)
[[0.26315072]
 [0.03634717]
 [0.02649979]
 [0.29885513]]
grad01:  [[-0.00699486 -0.01531808]
 [-0.00490175 -0.01070293]]
(2, 

In [49]:
# perceptron implementado na raça

import numpy as np
class Perceptron:
    def __init__(self, weights=None, bias=-1, activation_threshold=0.5, learning_rate=0.1, epochs=1000):
        if weights == None:
            self.weights = np.array([1, 1])
        else:
            self.weights = np.array(weights)
        self.bias = bias
        self.bias_hidden = np.random.uniform(size=(1, 2))
        self.bias_output = np.random.uniform(size=(1, 1))
        self.weights_hidden_input = np.random.uniform(size=(2, 2))
        self.weights_hidden_output = np.random.uniform(size=(2, 1))
        self.activation_threshold = activation_threshold
        self.learning_rate = learning_rate
        self.epochs = epochs

    def _heaviside(self, x):
        """
        Implementa a função delta de heaviside (famoso degrau)
        Essa é uma função de ativação possível para os nós da rede neural.
        """
        return np.where(x >= self.activation_threshold, 1, 0)

    def _sigmoid(self, x):
        """
        Implementa a função sigmoide
        Essa é uma função de ativação possível para os nós da rede neural.
        """
        return 1/(1 + np.exp(-x))

    def _der_sigmoid(self, x):
        """
        Implementa a derivada da função sigmoide
        """
        return x * (1 - x)


    def _activation(self, perceptron_output):
        """
        Implementação da função de ativação do perceptron
        Escolha uma das funções de ativação possíveis
        """
        return self._sigmoid(perceptron_output)

    def forward_pass(self, data):
        """
        Implementa a etapa de inferência (feedforward) do perceptron.
        """
        self.hidden_layer_input = np.dot(data, self.weights_hidden_input) + self.bias_hidden
        self.hidden_layer_output = self._activation(self.hidden_layer_input)

        self.output_layer_input = np.dot(self.hidden_layer_output, self.weights_hidden_output) + self.bias_output
        self.output_layer_output = self._activation(self.output_layer_input)

        return self.output_layer_output
    
    def loss_function(self, data, target):
        """
        Implementa a função de perda do perceptron, nesse caso, o erro médio quadrático
        """
        return 0.5 * (target - data) ** 2
    
    
    def backpropagation(self, data, target):
        """
        Implementa a etapa de backpropagation do perceptron a partir do foward pass.
        """
        output_error = self.loss_function(target, self.output_layer_output)
        output_gradient = output_error * self._der_sigmoid(self.output_layer_output)

        hidden_error = np.dot(output_gradient, self.weights_hidden_output.T)
        hidden_gradient = hidden_error * self._der_sigmoid(self.hidden_layer_output)

        self.weights_hidden_output += self.learning_rate * np.dot(self.hidden_layer_output.T, output_gradient)
        self.bias_output += self.learning_rate * np.sum(output_gradient, axis=0)

        self.weights_hidden_input += self.learning_rate * np.dot(data.T, hidden_gradient)
        self.bias_hidden += self.learning_rate * np.sum(hidden_gradient, axis=0)
    
    def train(self, data, target):
        """
        Treina o perceptron com os dados de entrada e saída esperada.
        """
        for _ in range(self.epochs):
            self.forward_pass(data)
            self.backpropagation(data, target)

    def predict(self, data):
        return self.forward_pass(data)
    
    

In [55]:
xor_nn = Perceptron()
xor_nn.train(train_data, target_xor)

# Faz previsões
predictions = xor_nn.predict(train_data)
print("Previsões para os dados de entrada do XOR:")
print(predictions)

Previsões para os dados de entrada do XOR:
[[0.99359278]
 [0.99574225]
 [0.99578184]
 [0.99670323]]
